### Simple GENAI App Using LANGCHAIN

In [61]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_PROJECT']=os.getenv("LANGCHAIN_PROJECT")


In [62]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://docs.djangoproject.com/en/5.2/")
docs=loader.load()
docs


[Document(metadata={'source': 'https://docs.djangoproject.com/en/5.2/', 'title': 'Django documentation | Django documentation | Django', 'description': '', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDjango documentation | Django documentation | Django\n\n\n\n\nSkip to main content\n\n\nDjango\nThe web framework for perfectionists with deadlines.\n\n\n\nSearch\n\n\n\nSubmit\n\n\n\n\n\nToggle theme (current theme: auto)\nToggle theme (current theme: light)\nToggle theme (current theme: dark)\nToggle Light / Dark / Auto color theme\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMenu\n\n\nMain navigation\n\n\nOverview\n\n\nDownload\n\n\nDocumentation\n\n\nNews\n\n\nCode\n\n\nIssues\n\n\nCommunity\n\n\nFoundation\n\n\n♥ Donate\n\n\n\n\nSearch\n\n\n\nSubmit\n\n\n\n\n\n\nToggle theme (current theme: auto)\nToggle theme (current theme: light)\nToggle theme (current theme: dark)\nToggle Light / Dark / Auto color theme\n\n\n\n\n\n\n\n

In [63]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
document=text_splitter.split_documents(docs)
document

[Document(metadata={'source': 'https://docs.djangoproject.com/en/5.2/', 'title': 'Django documentation | Django documentation | Django', 'description': '', 'language': 'en'}, page_content='Django documentation | Django documentation | Django\n\n\n\n\nSkip to main content\n\n\nDjango\nThe web framework for perfectionists with deadlines.\n\n\n\nSearch\n\n\n\nSubmit\n\n\n\n\n\nToggle theme (current theme: auto)\nToggle theme (current theme: light)\nToggle theme (current theme: dark)\nToggle Light / Dark / Auto color theme\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMenu\n\n\nMain navigation\n\n\nOverview\n\n\nDownload\n\n\nDocumentation\n\n\nNews\n\n\nCode\n\n\nIssues\n\n\nCommunity\n\n\nFoundation\n\n\n♥ Donate\n\n\n\n\nSearch\n\n\n\nSubmit\n\n\n\n\n\n\nToggle theme (current theme: auto)\nToggle theme (current theme: light)\nToggle theme (current theme: dark)\nToggle Light / Dark / Auto color theme\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDocumentation\n\n\n\n\n\n\n\n\n\n\nGetting Help\n\n\n\n\n\nel\n\n\nes\

In [70]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [65]:
from langchain_community.vectorstores import FAISS
vectordb=FAISS.from_documents(document,embeddings)

In [66]:
query="What is a Django model?"
data=vectordb.similarity_search(query)
data[0].page_content

'The model layer¶\nDjango provides an abstraction layer (the “models”) for structuring and\nmanipulating the data of your web application. Learn more about it below:\n\nModels:\nIntroduction to models |\nField types |\nIndexes |\nMeta options |\nModel class\nQuerySets:\nMaking queries |\nQuerySet method reference |\nLookup expressions\nModel instances:\nInstance methods |\nAccessing related objects\nMigrations:\nIntroduction to Migrations |\nOperations reference |\nSchemaEditor |\nWriting migrations\nAdvanced:\nManagers |\nRaw SQL |\nTransactions |\nAggregation |\nSearch |\nCustom fields |\nMultiple databases |\nCustom lookups |\nQuery Expressions |\nConditional Expressions |\nDatabase Functions\nOther:\nSupported databases |\nLegacy databases |\nProviding initial data |\nOptimize database access |\nPostgreSQL specific features'

In [67]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template("""Answer the following question based on the provided context:

<context>
{context}
</context>

""")

document_chain=create_stuff_documents_chain(llm, prompt)

document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Answer the following question based on the provided context:\n\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completio

In [77]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"What is Django model?",
    "context":[Document(page_content="What is Django model?")]
})

"A Django model is a Python class that represents and defines the structure of a database table in the Django web framework. Each model corresponds to a table in the database, and the attributes of the class represent the fields in that table. Django models are used to interact with the database, allowing developers to create, retrieve, update, and delete records easily using Python code. The framework automatically provides an abstraction layer over the database, making operations straightforward and reducing the need to write raw SQL queries. Models also support querying using Django's own ORM (Object-Relational Mapping), which makes data manipulation intuitive and Pythonic."

In [ ]:
retriever=vectordb.as_retriever()
from langchain_classic.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001884543B7C0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Answer the following question based on the provided context:\n\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
            | ChatOpe

In [ ]:
response=retrieval_chain.invoke({"input":"What is Django model?"})
response

{'input': 'What is Django model?',
 'context': [Document(id='1daa0711-bdcd-4fc8-b0c1-d031bdae7123', metadata={'source': 'https://docs.djangoproject.com/en/5.2/', 'title': 'Django documentation | Django documentation | Django', 'description': '', 'language': 'en'}, page_content='The model layer¶\nDjango provides an abstraction layer (the “models”) for structuring and\nmanipulating the data of your web application. Learn more about it below:\n\nModels:\nIntroduction to models |\nField types |\nIndexes |\nMeta options |\nModel class\nQuerySets:\nMaking queries |\nQuerySet method reference |\nLookup expressions\nModel instances:\nInstance methods |\nAccessing related objects\nMigrations:\nIntroduction to Migrations |\nOperations reference |\nSchemaEditor |\nWriting migrations\nAdvanced:\nManagers |\nRaw SQL |\nTransactions |\nAggregation |\nSearch |\nCustom fields |\nMultiple databases |\nCustom lookups |\nQuery Expressions |\nConditional Expressions |\nDatabase Functions\nOther:\nSupporte

In [ ]:
response['answer']

"What is the purpose of the “models” in Django's framework?\n\nDjango’s “models” serve as an abstraction layer for structuring and manipulating the data of a web application. They provide a way to define the data structure and schema, create and manage queries, and interact with the database in an organized manner, allowing developers to focus on the application logic without dealing directly with raw database operations."